In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\hardip thummar\Desktop\Book11.csv")

In [3]:
df2 = pd.read_csv(r"C:\Users\hardip thummar\Desktop\Book2.csv")

In [4]:
null_df = df.loc[pd.isnull(df['month']) == True]     # seperate dataframe of month column to be updated and month that
alloted_df = df.loc[pd.isnull(df['month']) == False]  # already has been updated

In [5]:
total_sal_month = {m : a for (m, a) in zip(df2['month'], df2['amt'])}     # create dictionary to total salary of month

In [6]:
# create dictionary of total alloted salary of respective month
if not len(df['month']) == df['month'].isna().sum():
    current_sal = {m : alloted_df.loc[df['month'] == m, 'amt'].sum() for m in df2['month']}
else:
    current_sal = {m : 0 for m in df2['month']}

In [7]:
# Update status of month , True if full else False
current_status = [True if current_sal[key] > total_sal_month[key] 
                  else False 
                  for key in total_sal_month.keys()]

In [8]:
# create dictionary of (index_value, month) and (month, index_value)
index_value = [(i, m) for (i, m) in zip(df2.index, df2['month'])]
index_value_2 = [(m, i) for (i, m) in zip(df2.index, df2['month'])]

index_value.extend(index_value_2)

index_dict = {a : b for a,b in index_value}

In [9]:
def assign_and_update():
    ''' Assigning month and update total salary and check if it is full '''
    df.at[index, 'month'] = current_index_month   # assign month to current index
    current_sal[df.at[index, 'month']] += current_index_sal    # update total salary
    if current_sal[current_index_month] > total_sal_month[current_index_month]:   # check if it is full
        current_status[index_dict[current_index_month]] = True

In [10]:
for a in null_df.itertuples():             # iterate over dataframe of month to be updated
    index = a[0]                           # assign index_value to index
    last_found_month = df[: index + 1].loc[df['code'] == df.loc[index, 'code'], 'month'].tail(2).head(1)  # search for last found month of a/c no.
    current_index_sal = null_df.at[index, 'amt']              # assign salary amount to current_index_sal
    if index == 0:
        df.at[index, 'month'] = str
        df.at[index, 'month'] = index_dict[current_status.index(False)]    # search for first available empty month
        current_sal[df.at[index, 'month']] += current_index_sal            # update total salary of that month by current salary
    else:
        if not pd.isnull(last_found_month.iloc[0]):         # if last month found
            last_found_month = last_found_month.iloc[0]
            check_from_month_index = index_dict[index_dict[index_dict[last_found_month] + 1]]
            current_index_month = index_dict[current_status[check_from_month_index : ].index(False) + check_from_month_index]
            assign_and_update()
        else:      # if last month not found
            current_index_month = index_dict[current_status.index(False)]
            assign_and_update()

In [11]:
df2['current_salary'] = current_sal.values()

In [12]:
df.to_csv(r"C:\Users\hardip thummar\Desktop\Book11.csv", index = False)
df2.to_csv(r"C:\Users\hardip thummar\Desktop\Book2.csv", index = False)